In [9]:
import pandas as pd
from datetime import datetime, timezone, timedelta
import csv
import os
import requests
import json
import time

In [10]:
# kr_high = pd.read_csv('data/01_puuids/kr_puuids_high_tiers.csv')

In [11]:
# os.mkdir('data/02_match-v5')

In [12]:
kst = timezone(timedelta(hours=9))
startTime = datetime(2025, 1, 9, 7, 0, 0, tzinfo=kst)
print(int(startTime.timestamp()))

1736373600


In [13]:
# 2025 1월 9일 07:00 (25.1.1패치 적용 시간 KST)
startTime_unix = int(startTime.timestamp())

In [14]:
def unix_conv(unix_time):
    dt = datetime.datetime.utcfromtimestamp(unix_time)
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt

In [18]:
API_KEY="RGAPI-ec13736b-acd7-4c71-883c-b94e97c40c13"
HEADERS = {"X-Riot-Token": API_KEY}

REGION_MAPPING = {
    'KR': 'asia', 'JP1': 'asia',
    'BR1': 'americas', 'NA1': 'americas', 'LA1': 'americas', 'LA2': 'americas',
    'EUN1': 'europe', 'EUW1': 'europe', 'TR1': 'europe', 'RU': 'europe', 'ME1': 'europe',
    'OC1': 'sea', 'SG2': 'sea', 'TW2': 'sea', 'VN2': 'sea'
}

OUTPUT_FILE = (r"data/02_match-v5/br1_collected_match_ids.csv")
PROGRESS_FILE = (r"data/02_match-v5/processed_puuids.txt")  # 수집 완료된 유저 저장용
SAVE_INTERVAL = 10

def get_matches_for_one_user(puuid, platform, total_count=1000):
    routing = REGION_MAPPING.get(platform.upper(), 'asia')
    url = f"https://{routing}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    
    user_match_ids = []
    for start_index in range(0, total_count, 100):
        params = {
            "api_key" : API_KEY,
            # "queue": 420,
            "startTime" : startTime_unix,
            "start": start_index, 
            "count": 100}
        try:
            response = requests.get(url, headers=HEADERS, params=params)
            if response.status_code == 200:
                fetched_ids = response.json()
                user_match_ids.extend(fetched_ids)
                if len(fetched_ids) < 100: break # 더 이상 데이터 없음
            elif response.status_code == 429:
                print("!! Rate Limit! 10초 대기...")
                time.sleep(8.8)
                continue # 현재 구간 다시 시도
            else:
                break
            time.sleep(1.2) # Personal Key 속도 준수
        except Exception as e:
            print(f"오류: {e}")
            break
    return user_match_ids

def save_to_csv(match_id_set, file_name):
    """
    수집된 매치 ID 세트를 CSV에 저장 (중복 제거 포함)
    """
    new_df = pd.DataFrame(list(match_id_set), columns=['match_id'])
    
    # 파일이 없으면 새로 만들고(header 포함), 있으면 이어 쓰기(header 제외)
    if not os.path.exists(file_name):
        new_df.to_csv(file_name, index=False, mode='w', encoding='utf-8')
    else:
        new_df.to_csv(file_name, index=False, mode='a', header=False, encoding='utf-8')
    
    print(f"--- [중간 저장 완료] {len(new_df)}개의 새로운 매치 ID 저장됨 ---")

def collect_with_checkpoints(df_dict):
    current_session_matches = set() # 현재 세션에서의 중복 방지용
    
    for platform, df in df_dict.items():
        print(f"\n🚀 {platform} 지역 수집 시작...")
        
        for idx, puuid in enumerate(df['puuid']):
            m_ids = get_matches_for_one_user(puuid, platform)
            current_session_matches.update(m_ids)
            
            # 중간 저장 로직: SAVE_INTERVAL(10명) 마다 실행
            if (idx + 1) % SAVE_INTERVAL == 0:
                print(f"[{idx+1}/{len(df)}] 진행 중...", end=' ')
                save_to_csv(current_session_matches, OUTPUT_FILE)
                # 저장 후 세션을 비워주면 메모리 부담이 줄어듭니다.
                current_session_matches.clear() 

        # 지역 하나가 끝나면 남은 데이터 저장
        if current_session_matches:
            save_to_csv(current_session_matches, OUTPUT_FILE)
            current_session_matches.clear()

    # 마지막으로 전체 파일에서 혹시 모를 중복 제거
    final_cleanup(OUTPUT_FILE)

def final_cleanup(file_name):
    print("\n🧹 전체 데이터 중복 제거 작업 시작...")
    df = pd.read_csv(file_name)
    before_count = len(df)
    df.drop_duplicates(subset=['match_id'], inplace=True)
    df.to_csv(file_name, index=False)
    print(f"✅ 최종 완료! ({before_count} -> {len(df)}개)")

# 실행
# collect_with_checkpoints(df_dict)

def load_processed_puuids():
    """이미 수집 완료된 유저 목록을 불러옵니다."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            # 한 줄씩 읽어서 세트에 저장 (중복 제거 및 빠른 검색용)
            return set(line.strip() for line in f)
    return set()

def save_progress(puuid_list):
    """수집 완료된 유저 목록을 파일에 추가합니다."""
    with open(PROGRESS_FILE, 'a') as f:
        for puuid in puuid_list:
            f.write(f"{puuid}\n")

def collect_with_resume(df_dict):
    # 1. 작업 완료 명단 로드
    processed_puuids = load_processed_puuids()
    print(f"📊 기존 작업 기록 확인: {len(processed_puuids)}명의 유저는 이미 수집되었습니다.")

    current_session_matches = set()
    newly_processed_this_session = [] # 이번 세션에서 새로 완료한 유저들

    for platform, df in df_dict.items():
        print(f"\n🚀 {platform} 지역 수집 시작...")
        
        for idx, puuid in enumerate(df['puuid']):
            # 2. 건너뛰기 로직
            if puuid in processed_puuids:
                continue 

            # 3. 데이터 수집
            m_ids = get_matches_for_one_user(puuid, platform)
            current_session_matches.update(m_ids)
            newly_processed_this_session.append(puuid)
            
            # 4. 중간 저장 (SAVE_INTERVAL 마다)
            if len(newly_processed_this_session) >= SAVE_INTERVAL:
                print(f"[{idx+1}/{len(df)}] 저장 중...", end=' ')
                save_to_csv(current_session_matches, OUTPUT_FILE)
                save_progress(newly_processed_this_session) # 유저 명단도 저장
                
                # 메모리 비우기
                current_session_matches.clear()
                processed_puuids.update(newly_processed_this_session)
                newly_processed_this_session = []

        # 지역 종료 후 남은 데이터 처리
        if newly_processed_this_session:
            save_to_csv(current_session_matches, OUTPUT_FILE)
            save_progress(newly_processed_this_session)
            current_session_matches.clear()
            newly_processed_this_session = []

    print("\n✨ 모든 지역 수집 프로세스가 종료되었습니다.")

## EXE

In [19]:
br1_high=pd.read_csv(r'D:\바탕화면\데이터 분석\프로젝트\04 최종 프로젝트\Project_Sparta\0_data\1_br1_csv\br1_puuids_high_tiers.csv')

In [20]:
df_dict_high_tiers = {
    "BR1": br1_high,
}

In [21]:
collect_with_resume(df_dict_high_tiers)

📊 기존 작업 기록 확인: 6030명의 유저는 이미 수집되었습니다.

🚀 BR1 지역 수집 시작...
[6040/6167] 저장 중... --- [중간 저장 완료] 8574개의 새로운 매치 ID 저장됨 ---
[6050/6167] 저장 중... --- [중간 저장 완료] 7393개의 새로운 매치 ID 저장됨 ---
[6060/6167] 저장 중... --- [중간 저장 완료] 7169개의 새로운 매치 ID 저장됨 ---
[6070/6167] 저장 중... --- [중간 저장 완료] 7068개의 새로운 매치 ID 저장됨 ---
[6080/6167] 저장 중... --- [중간 저장 완료] 8443개의 새로운 매치 ID 저장됨 ---
[6090/6167] 저장 중... --- [중간 저장 완료] 7878개의 새로운 매치 ID 저장됨 ---
[6100/6167] 저장 중... --- [중간 저장 완료] 7152개의 새로운 매치 ID 저장됨 ---
[6110/6167] 저장 중... --- [중간 저장 완료] 7285개의 새로운 매치 ID 저장됨 ---
[6120/6167] 저장 중... --- [중간 저장 완료] 7084개의 새로운 매치 ID 저장됨 ---
[6130/6167] 저장 중... --- [중간 저장 완료] 5726개의 새로운 매치 ID 저장됨 ---
[6140/6167] 저장 중... --- [중간 저장 완료] 7381개의 새로운 매치 ID 저장됨 ---
[6150/6167] 저장 중... --- [중간 저장 완료] 7969개의 새로운 매치 ID 저장됨 ---
[6160/6167] 저장 중... --- [중간 저장 완료] 8764개의 새로운 매치 ID 저장됨 ---
--- [중간 저장 완료] 5765개의 새로운 매치 ID 저장됨 ---

✨ 모든 지역 수집 프로세스가 종료되었습니다.


In [ ]:
import pandas as pd
import requests
import time
import os

# 1. 설정 (기존 노트북 변수 활용)
# API_KEY는 기존 셀에 입력된 것을 그대로 사용하거나 아래에 다시 입력하세요.
# API_KEY = "내_API_KEY" 
HEADERS = {"X-Riot-Token": API_KEY}

# 서버별 매치 지역(Regional) 라우팅 설정
# BR1, LA1 -> AMERICAS / TR1 -> EUROPE / VN2 -> ASIA
routing_map = {
    "br1": "americas",
    "la1": "americas",
    "tr1": "europe",
    "vn2": "asia"
}

platforms = ['br1', 'la1', 'tr1', 'vn2']

for pf in platforms:
    print(f"\n>>> [{pf.upper()}] 매치 ID 수집 시작")
    
    # (1) 샘플링된 PUUID 파일 읽기
    file_path = f"{pf}_combined_high_sampled.csv"
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        continue
        
    df_sample = pd.read_csv(file_path)
    puuids = df_sample['puuid'].unique()
    
    region = routing_map[pf]
    all_match_ids = set() # 중복 방지를 위해 set 사용

    # (2) 각 PUUID별로 최근 20게임 매치 ID 호출
    for i, puuid in enumerate(puuids):
        # 2분당 100회(개발키 기준) 제한을 고려한 속도 조절
        if i % 10 == 0 and i > 0:
            print(f"  - 진행률: {i}/{len(puuids)} 명 완료...")
            
        url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20"
        
        try:
            response = requests.get(url, headers=HEADERS)
            
            if response.status_code == 200:
                match_ids = response.json()
                all_match_ids.update(match_ids)
            elif response.status_code == 429:
                print("!! Rate Limit 발생! 10초 대기...")
                time.sleep(10)
            else:
                print(f"!! 에러 발생 ({response.status_code})")
                
            time.sleep(0.05) # 미세한 지연으로 안정성 확보
            
        except Exception as e:
            print(f"!! 연결 오류: {e}")

    # (3) 결과 저장
    match_df = pd.DataFrame(list(all_match_ids), columns=['matchId'])
    output_path = f"{pf}_sampled_match_ids.csv"
    match_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ [{pf.upper()}] 수집 완료: 총 {len(match_df)}개의 고유 매치 ID 저장")

print("\n✨ 모든 서버의 매치 ID 수집이 완료되었습니다!")